In [31]:
import pandas as pd

artist_data = pd.read_csv('datasets/ydata-ymusic-user-artist-rating.csv')
artist_name_data = pd.read_csv('datasets/ydata-ymusic-artist-names.csv')


In [32]:
#Data preprocessing
def data_preprocessing(artist_data):
    
    #to show popular artist
    avg_rating = pd.DataFrame(artist_data.groupby('artist_id')['rating'].mean().reset_index())
    avg_rating.sort_values('rating', ascending = False).head(20)
    
    #creating pivot table
    user_rating = pd.pivot_table(data=artist_data, values='rating', index='user_id', columns='artist_id')
    
    return user_rating

In [33]:
def data_cleaning(recommended_artist):
    
     #data cleaning and sorting
    recommended_artist_corr = pd.DataFrame(recommended_artist, columns=['Correlation'])
    recommended_artist_corr.dropna(inplace=True)
    recommended_artist_corr = recommended_artist_corr.sort_values('Correlation', ascending=False).reset_index()
    
    return recommended_artist_corr

In [34]:
def get_top_n_artist(recommended_artist_corr, n):
    
    #get top n artist list and artist names
    artist_list = recommended_artist_corr.head(n)['artist_id'].tolist()
    artist_names = artist_name_data[artist_name_data['artist_id'].isin(artist_list)]
    
    return artist_names

In [35]:
def collaborative_filtering(artist_id, n = 5):
    
    #calling data preprocessing
    user_rating = data_preprocessing(artist_data)
    
    #artist_id 
    #see artists rated by user
    artist_user_rating = user_rating[artist_id]
    
    #applying correlation for the entire user rating dataframe
    #Collaborative Filtering Song Recommendation
    #Pearson Correlation
    recommended_artist = user_rating.corrwith(artist_user_rating)
    
    #data cleaning and sorting
    recommended_artist_corr = data_cleaning(recommended_artist)
    
    #get top n artist list and artist names
    artist_names = get_top_n_artist(recommended_artist_corr, n)
    
    return artist_names

In [36]:
recommended_artist_name = collaborative_filtering(1058037, 5)
recommended_artist_name

/opt/homebrew/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,artist_id,artist
15486,1015911,Lords Of Acid
17287,1017742,Melvins
34356,1035342,Borknagar
39172,1040238,Mindless Self Indulgence
39633,1040701,Speak No Evil
